In [1]:
#BigdataFinal: 6304442324:AAHrPrvuwXjxgAYQE17QJkWbZbvz1cVC-tQ

In [2]:
!pip install telegram

  Preparing metadata (setup.py) ... done
  Created wheel for telegram: filename=telegram-0.0.1-py3-none-any.whl size=1295 sha256=8450665f64e25aff9f01614266b9c23bb0c02c475149b25d5c18ae49b4299ea9
  Stored in directory: /root/.cache/pip/wheels/f3/f2/16/dc6d6c6c7e0422206fec62833039a2b4082de3b85e55d32b52
Successfully built telegram


In [3]:
!pip install python-telegram-bot==12.8


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.4/375.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 68.9 MB/s eta 0:00:00


In [4]:
FRESH_START = False
my_Tel_API = '6304442324:AAHrPrvuwXjxgAYQE17QJkWbZbvz1cVC-tQ'
if FRESH_START:
  LoadUsersPath = '/content/drive/MyDrive/Final Project BigData/telegrambotusers.csv'
  ReceivedMsgsPath = '/content/drive/MyDrive/Final Project BigData/NewsReceoved.csv'
else:
  LoadUsersPath = '/content/drive/MyDrive/Final Project BigData/telegrambotusers_Reload.csv'
  ReceivedMsgsPath = '/content/drive/MyDrive/Final Project BigData/NewsReceoved_Reload.csv'


LoadUsersPathSaveLocation = '/content/drive/MyDrive/Final Project BigData/telegrambotusers_Reload.csv'
ReceivedMsgsPathSaveLocation = '/content/drive/MyDrive/Final Project BigData/NewsReceoved_Reload.csv'
StreamDataPath = '/content/drive/MyDrive/Final Project BigData/StreamData_preprocessed.csv'

In [5]:
import pandas as pd
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from datetime import datetime
import time
import requests
import threading
import time
from collections import Counter

UsersDataDF = pd.read_csv(LoadUsersPath)
NewsReceived = pd.read_csv(ReceivedMsgsPath, encoding='utf-8-sig')

TotalReceivedNews = len(NewsReceived)
FearAndGreedIndex = 50
FearAndGreedIndex_ML = 50
Score_List=[]
ML_Score_List=[]
Hashtag_List=[]
Category_List=[]

In [6]:
print(UsersDataDF)
print(f'news len from last run: {len(NewsReceived)}')

      userid  broadcast
0  549737016       True
news len from last run: 179


In [7]:
def get_crypto_prices():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum,ripple,cardano,dogecoin,binancecoin&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()
    bitcoin_price = data['bitcoin']['usd']
    ethereum_price = data['ethereum']['usd']
    ripple_price = data['ripple']['usd']
    cardano_price = data['cardano']['usd']
    dogecoin_price = data['dogecoin']['usd']
    bnb_price = data['binancecoin']['usd']
    return bitcoin_price, ethereum_price, ripple_price, cardano_price, dogecoin_price, bnb_price

bitcoin_price, ethereum_price, ripple_price, cardano_price, dogecoin_price, bnb_price = get_crypto_prices()
print("Bitcoin price in USD:", bitcoin_price)
print("Ethereum price in USD:", ethereum_price)
print("Ripple price in USD:", ripple_price)
print("Cardano price in USD:", cardano_price)
print("Dogecoin price in USD:", dogecoin_price)
print("Binance Coin price in USD:", bnb_price)

def update_prices():
    global bitcoin_price, ethereum_price, ripple_price, cardano_price, dogecoin_price, bnb_price
    while True:
        bitcoin_price, ethereum_price, ripple_price, cardano_price, dogecoin_price, bnb_price = get_crypto_prices()
        time.sleep(60)
thread = threading.Thread(target=update_prices)
thread.start()

Bitcoin price in USD: 30095
Ethereum price in USD: 1905.35
Ripple price in USD: 0.733476
Cardano price in USD: 0.310388
Dogecoin price in USD: 0.069109
Binance Coin price in USD: 242.74


In [8]:
def calculateNewFearAndIndex(numbers):
    # based on last 20 news.
    if len(numbers) == 0:
        return 50
    if len(numbers) < 20:
        average = sum(numbers) / len(numbers)
    else:
        last_20 = numbers[-20:]
        average = sum(last_20) / len(last_20)
    return average

def classify_number(num):
    if num < 0:
        num = abs(num)
    if num < 20:
        return 'بسیار منفی🔴'
    elif num < 40:
        return 'منفی🔴'
    elif num < 60:
        return 'بی تاثیر'
    elif num < 80:
        return 'مثبت🟢'
    else:
        return 'بسیار مثبت🟢'

def sendToAllBroadcastUsers(updater,msg):
  global UsersDataDF
  for index, row in UsersDataDF.iterrows():
        if row['broadcast']:
            user_id = row['userid']
            updater.bot.send_message(chat_id=user_id, text=msg)

In [11]:
def getTop3Hashtags():
  global Hashtag_List
  hashtags = []
  for item in Hashtag_List:
      if item != '[]':
          hashtags.extend(eval(item))

  hashtag_counts = Counter(hashtags)
  top_3_hashtags = hashtag_counts.most_common(3)

  return top_3_hashtags

def start(update, context):
    global UsersDataDF
    user_id = update.effective_chat.id
    if UsersDataDF['userid'].eq(user_id).any():
        context.bot.send_message(chat_id=user_id, text="با سلام. شما قبلا از این بات استفاده کرده اید. خوش برگشتید!")
    else:
        new_row = {'userid': user_id, 'broadcast': True}
        UsersDataDF = UsersDataDF.append(new_row, ignore_index=True)
        context.bot.send_message(chat_id=user_id, text="با سلام. به بات تحلیل خبری بیگ دیتا خوش آمدید! اگر تمایلی به دریافت خبرهای جدید ندارید دستور /broadcast off را ارسال کنید.")

def summary(update, context):
  global FearAndGreedIndex,FearAndGreedIndex_ML,UsersDataDF
  Top3Hashtags = getTop3Hashtags()
  user_id = update.effective_chat.id
  if not UsersDataDF['userid'].eq(user_id).any():
        new_row = {'userid': user_id, 'broadcast': True}
        UsersDataDF = UsersDataDF.append(new_row, ignore_index=True)

  part1 = f'💵قیمت ارز دیجیتال‌های برتر:\n💲BTC/USDT: {bitcoin_price}\
                  \n💲ETH/USDT: {ethereum_price}\
                  \n💲XRP/USDT: {ripple_price}\
                  \n💲ADA/USDT: {cardano_price}\
                  \n💲DOGE/USDT: {dogecoin_price}\
                  \n💲BNB/USDT: {bnb_price}\n\n'
  if FearAndGreedIndex<50: emoji1 = '🔴'
  else: emoji1='🟢'
  if FearAndGreedIndex_ML<50: emoji2 = '🔴'
  else: emoji2='🟢'
  part2 = f'امتیاز شاخص ترس و طمع: {FearAndGreedIndex}{emoji1}\nامتیاز شاخص ترس و طمع با استفاده از هوش مصنوعی:{FearAndGreedIndex_ML}{emoji2}\n'
  if len(Top3Hashtags) == 0:
    part3 = ''
  elif len(Top3Hashtags) == 1:
      part3 = f'📢بیشتر خبر‌های منتشر شده درمورد هشتگ‌های {Top3Hashtags[0][0]} با {Top3Hashtags[0][1]} تکرار است.\n'
  elif len(Top3Hashtags) == 2:
    part3 = f'📢بیشتر خبر‌های منتشر شده درمورد هشتگ‌های {Top3Hashtags[0][0]} با {Top3Hashtags[0][1]} تکرار و {Top3Hashtags[1][0]} با {Top3Hashtags[1][1]} تکرار است.\n'
  else:
    part3 = f'📢بیشتر خبر‌های منتشر شده درمورد هشتگ‌های {Top3Hashtags[0][0]} با {Top3Hashtags[0][1]} تکرار و {Top3Hashtags[1][0]} با {Top3Hashtags[1][1]} تکرار و {Top3Hashtags[2][0]} با {Top3Hashtags[2][1]} تکرار است.\n'
  part4 = f'تعداد کل خبر‌های استریم شده به بات: {TotalReceivedNews}\n\nجهت مشاهده خبر‌ها ایندکس خبر مورد نظر را به \n/getnews \nبفرستید'
  msg = part1 + part2 + part3 + part4
  context.bot.send_message(chat_id=update.effective_chat.id, text=msg)


def broadcast(update, context):
    global UsersDataDF
    user_id = update.effective_chat.id
    if not UsersDataDF['userid'].eq(user_id).any():
        new_row = {'userid': user_id, 'broadcast': True}
        UsersDataDF = UsersDataDF.append(new_row, ignore_index=True)
    args = context.args
    if len(args) == 1:
        param = args[0]
        if param == 'off':
          UsersDataDF.loc[UsersDataDF['userid'] == user_id, 'broadcast'] = False
          response_text = f"عملیات با موفقیت انجام شد. شما با یوزر آیدی ({user_id}) خبر جدیدی دریافت نخواهید کرد."
          context.bot.send_message(chat_id=user_id, text=response_text)
        elif param == 'on':
          UsersDataDF.loc[UsersDataDF['userid'] == user_id, 'broadcast'] = True
          response_text = f"عملیات با موفقیت انجام شد. شما با یوزر آیدی ({user_id}) خبرهای جدید را دریافت خواهید کرد."
          context.bot.send_message(chat_id=user_id, text=response_text)
        else:
          response_text = "ورودی نادرست است. لطفا فقط مقدار on یا off را ارسال کنید."
          context.bot.send_message(chat_id=user_id, text=response_text)
    elif len(args) == 0:
        Status = UsersDataDF.loc[UsersDataDF['userid'] == user_id, 'broadcast'].values[0]
        context.bot.send_message(chat_id=user_id, text=f"وضعیت Broadcast اکانت شما در حالت {Status} قرار دارد. برای فعال سازی دستور\n/broadcast on\n و برای غیرفعال‌سازی دستور\n/broadcast off\n را اجرا کنید.")
    else:
        context.bot.send_message(chat_id=user_id, text="لطفا فقط یک پارامتر ارسال کنید. پارامتر‌های قابل قبول: off یا on می‌باشد.")


def echo(update, context):
    global UsersDataDF
    user_id = update.effective_chat.id
    if not UsersDataDF['userid'].eq(user_id).any():
        new_row = {'userid': user_id, 'broadcast': True}
        UsersDataDF = UsersDataDF.append(new_row, ignore_index=True)
    response_text = "لطفا از دستورات قابل پشتیبانی توسط بات استفاده کنید. دستورات قابل پشتیبانی:\n1./broadcast (off یا on)\n2./summary\n3./getnews"
    context.bot.send_message(chat_id=update.effective_chat.id, text=response_text)

def getNews(update, context):
  global NewsReceived,UsersDataDF
  user_id = update.effective_chat.id
  if not UsersDataDF['userid'].eq(user_id).any():
      new_row = {'userid': user_id, 'broadcast': True}
      UsersDataDF = UsersDataDF.append(new_row, ignore_index=True)
  args = context.args
  if len(args) == 0:
    context.bot.send_message(chat_id=user_id, text=f"با این دستور می‌توانید تمامی اخبار موجود در ربات را دوباره مشاهده کنید. تعداد کل خبر‌های موجود در بات: {TotalReceivedNews}\n برای مشاهده خبر مورد نظر ایندکس آن را به این دستور بفرستید. به عنوان مثال:\n/getnews 1")
  elif len(args) == 1:
    try:
        param = int(args[0])
    except ValueError as e:
        context.bot.send_message(chat_id=user_id, text=f"لطفا یک عدد را به عنوان ایندکس بفرستید. مثال:\n/getnews 1 \n")
    param = int(param - 1)
    if param > len(NewsReceived) or param<0:
        context.bot.send_message(chat_id=user_id, text=f"همچین خبری وجود ندارد. یا ممکن است عدد منفی فرستاده باشید. لطفا یک عدد کوچک‌تر از {(len(NewsReceived)+1)} بفرستید.")
    else:
        row = NewsReceived.iloc[param]
        msg = f'متن خبر شماره {(param+1)}:\n\n{row["clean_message"]}\n\nهشتگ‌ها: {row["hashtags"]}\nتاثیر این خبر: {classify_number(row["score"])}\nتاثیر این خبر با استفاده از مدل یادگیری ماشین: {classify_number(row["ML_score"])}\nدسته‌بندی:{row["category"]}\nزمان انتشار خبر: {datetime.strptime(row["date"], "%Y-%m-%d %H:%M:%S%z").strftime("%H:%M:%S %d-%m-%Y")}\n'
        context.bot.send_message(chat_id=user_id, text=msg)
  else:
    context.bot.send_message(chat_id=user_id, text=f"این بات درحد یک پروژه دانشگاهی می‌باشد. لطفا بیشتر از یک ایندکس نفرستید.")



In [ ]:
running = True

def main():
    global running
    updater = Updater(token=my_Tel_API, use_context=True)

    dispatcher = updater.dispatcher

    start_handler = CommandHandler('start', start)
    dispatcher.add_handler(start_handler)

    login_handler = CommandHandler('broadcast', broadcast)
    dispatcher.add_handler(login_handler)

    broadcast_handler = CommandHandler('summary', summary)
    dispatcher.add_handler(broadcast_handler)

    broadcast_handler = CommandHandler('getnews', getNews)
    dispatcher.add_handler(broadcast_handler)

    echo_handler = MessageHandler(Filters.text, echo)
    dispatcher.add_handler(echo_handler)

    updater.start_polling()

    StreamNewsThread = threading.Thread(target=Stream_200_News, args=(updater,))
    StreamNewsThread.start()

    updater.idle()


    # Bot Stopped. save the Dataframes and stop streamdata streaming.
    running = False
    print("Telegram bot stopped. wait for StreamNewsThread to stop.")
    NewsReceived.reset_index(drop=True, inplace=True)
    StreamNewsThread.join()

    UsersDataDF.to_csv(LoadUsersPathSaveLocation, index=False)
    NewsReceived.to_csv(ReceivedMsgsPathSaveLocation, index=False, encoding='utf-8-sig')
    print('saved UsersDataDF and NewsReceived')

def Stream_200_News(updater):
    global TotalReceivedNews, FearAndGreedIndex, FearAndGreedIndex_ML
    global ML_Score_List, Score_List, Hashtag_List, Category_List
    global StreamDataPath, NewsReceived
    global running
    StreamData = pd.read_csv(StreamDataPath, encoding='utf-8-sig')
    for index, row in StreamData.iterrows():
      time.sleep(30)
      if running==False: break
      #NewsReceived = NewsReceived.append(row)
      NewsReceived = pd.concat([NewsReceived, pd.DataFrame(row).T], ignore_index=True)
      TotalReceivedNews = TotalReceivedNews + 1
      Score_List.append(row["score"])
      ML_Score_List.append(row["ML_score"])
      Hashtag_List.append(row["hashtags"])
      Category_List.append(row["category"])

      FearAndGreedIndex = calculateNewFearAndIndex(Score_List)
      FearAndGreedIndex_ML = calculateNewFearAndIndex(ML_Score_List)
      msg = f'خبر جدیدی از کانال @NEWS_FUNDA دریافت شد. متن خبر:\n\n{row["clean_message"]}\n\nهشتگ‌ها: {row["hashtags"]}\nتاثیر این خبر: {classify_number(row["score"])}\nتاثیر این خبر با استفاده از مدل یادگیری ماشین: {classify_number(row["ML_score"])}\nدسته‌بندی:{row["category"]}\nزمان انتشار خبر: {datetime.strptime(row["date"], "%Y-%m-%d %H:%M:%S%z").strftime("%H:%M:%S %d-%m-%Y")}\n\nبرای مشاهده لحظه‌ای قیمت‌ها و تاثیر 20 خبر اخیر بر روی بازار دستور\n/summary \nرا اجرا کنید. برای غیر فعال‌سازی نمایش خبر‌های جدید دستور\n/broadcast off \nرا اجرا کنید. همچنین این خبر در بات ذخیره شده است و برای مشاهده دوباره آن می‌توانید از دستور\n/getnews {TotalReceivedNews}\nاستفاده کنید.'
      sendToAllBroadcastUsers(updater,msg)

main()

